In [2]:
import gym
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [4]:
envName = 'CartPole-v1'
env = gym.make(envName)

In [171]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:0 Score:24.0
Episode:1 Score:11.0
Episode:2 Score:28.0
Episode:3 Score:19.0
Episode:4 Score:11.0


In [172]:
logPath = os.path.join("Training", 'Logs')
print(logPath)

Training/Logs


In [173]:
env = gym.make(envName)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = logPath)

Using cpu device


/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [174]:
model.learn(total_timesteps = 40000)

Logging to Training/Logs/PPO_13
-----------------------------
| time/              |      |
|    fps             | 2801 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2082        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008132288 |
|    clip_fraction        | 0.106       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00027    |
|    learning_rate        | 0.0003      |
|    loss                 | 9.56        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0166     |
|    value_loss           | 59.5        |
-----------------------------------------
--

In [8]:
PPO_Path = os.path.join("Training", "Saved Models", "PPO_Model_Cartpole")
model.save(PPO_Path)

In [9]:
del model
model = PPO.load(PPO_Path, env = env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [177]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(500.0, 0.0)

In [10]:
env = gym.make(envName)
episodes = 5
obs = env.reset()
action, _ = model.predict(obs, deterministic=True)

for episode in range(episodes):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:0 Score:500.0
Episode:1 Score:500.0
Episode:2 Score:500.0
Episode:3 Score:500.0
Episode:4 Score:500.0


In [179]:
trainingLogPath = os.path.join(logPath, "PPO_12")

# !tensorboard --logdir={trainingLogPath}

In [13]:
# Adding a callback to the training Stage
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os


In [14]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [15]:
env = gym.make(envName)
env = DummyVecEnv([lambda: env])

/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [16]:
stopCallback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
evalCallback = EvalCallback(env, callback_on_new_best = stopCallback, eval_freq = 10000, best_model_save_path = save_path, verbose = 1)

In [17]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cpu device


In [18]:
model.learn(total_timesteps=20000, callback=evalCallback)

Logging to Training/Logs/PPO_14
-----------------------------
| time/              |      |
|    fps             | 3014 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2230        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008028299 |
|    clip_fraction        | 0.0849      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00581    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.97        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.012      |
|    value_loss           | 60.7        |
-----------------------------------------
--

/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=409.60 +/- 112.79
Episode length: 409.60 +/- 112.79
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 410         |
|    mean_reward          | 410         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.008366033 |
|    clip_fraction        | 0.0758      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.612      |
|    explained_variance   | 0.276       |
|    learning_rate        | 0.0003      |
|    loss                 | 32.2        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0155     |
|    value_loss           | 64.5        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 409.60  is above the threshold 190


In [19]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


In [20]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(434.6, 110.1391846710334)

In [21]:
# Changing Policies

net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [22]:
model.learn(total_timesteps=20000, callback=evalCallback)

-----------------------------
| time/              |      |
|    fps             | 2636 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1686        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014650216 |
|    clip_fraction        | 0.223       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.00269    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.99        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0262     |
|    value_loss           | 21          |
-----------------------------------------
----------------------------------

In [23]:
from stable_baselines3 import DQN

model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [24]:
model.learn(total_timesteps=20000, callback=evalCallback)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.965    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 8664     |
|    time_elapsed     | 0        |
|    total_timesteps  | 73       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.938    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 9980     |
|    time_elapsed     | 0        |
|    total_timesteps  | 130      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.91     |
| time/               |          |
|    episodes         | 12       |
|    fps              | 11131    |
|    time_elapsed     | 0        |
|    total_timesteps  | 189      |
----------------------------------
------------------------

In [25]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

model.save(dqn_path)

model = DQN.load(dqn_path, env=env)

/home/joao/.local/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


In [31]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.6, 0.4898979485566356)

In [33]:
env = gym.make(envName)
episodes = 5
obs = env.reset()
action, _ = model.predict(obs, deterministic=True)

for episode in range(episodes):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:0 Score:9.0
Episode:1 Score:10.0
Episode:2 Score:9.0
Episode:3 Score:8.0
Episode:4 Score:10.0
